In [5]:
from metric.compute_metric import compute_clear
from metric.read_file import read_file
from utils import make_pred, save_data, iter_frame, body_barycenter, head_barycenter
from utils import Point
from utils import Corp, CorpCol
from utils import Identity
from utils import mult_load
from utils import pretraitement
import numpy as np
PATH = "data/Scenario03-01/"


In [6]:
# data_ref = read_file(PATH+"groundTruth.txt")[630:680]
# row_data = np.loadtxt(PATH+"detections.txt")[630:680]
data_ref = read_file(PATH+"groundTruth.txt")
row_data = np.loadtxt(PATH+"augmented_data.txt")

print(data_ref.shape)
print(row_data.shape)

(7717, 39)
(7669, 45)


(7717, 39)
(7669, 45)


In [5]:
# coding=utf-8
from random import random, randint
from munkres import Munkres, print_matrix

def make_alea(proba_trans, dist, conf=None):
    """
    dist : fonction de distance entre identitée et corp
    :param proba_trans: 
    :param dist: 
    :param conf: 
    :return: 
    """
    def alea_par_func(data):
        save_pos = dict()
        parle = 0
        for frame in iter_frame(data):
            nb_personne = frame.shape[0]

            if random() < proba_trans:
                parle = randint(0, nb_personne)

            save_pos = identity_assignation(frame, save_pos, conf)

            for i, _ in enumerate(frame):
                cp = Corp(_)
                # if save_pos[cp] == 5:
                    # print(save_pos)
                yield (save_pos[cp], save_pos[cp] == parle)
    return alea_par_func


def identity_assignation(frame, oldPos, conf, dist):
    """
    dist : fonction de distance entre identitée et corp
    :param frame: 
    :param oldPos: 
    :param conf: 
    :param dist: 
    :return: 
    """
    to_assin = []
    new_pos = []
    rep = dict()
    for line in frame:
        to_assin.append(Corp(line))
        new_pos.append(0)
        
    new_pos_traite = []
    old_pos_traite = []
    
    m = Munkres()
    matrice = []
    old_pos_order = oldPos.keys()
    for old in oldPos:
        l =[]
        for new in to_assin:
            l.append(dist((old, new)))
        matrice.append(l)
    # print("old pos oder" + str(old_pos_order))
    # print("to_assin" + str(to_assin))
    # print(matrice)
    # 
    # 
    # if len(matrice) != 0 and (max([max(l)for l in matrice])-min([min(l)for l in matrice]) < 100):
    #     print (frame)
    #     print("max " + str(max([max(l)for l in matrice])))
    #     print("min " + str(min([min(l)for l in matrice])))

    
    if len(matrice) != 0:
        indice = m.compute(matrice)
        i_min = min((len(old_pos_order), len(to_assin)))
        indice = indice[:i_min] # on suprime le 0 pading
        for (old, new) in indice:
            id_old = old_pos_order[old]
            pt_new = to_assin[new]
            old_pos_traite.append(id_old)
            new_pos_traite.append(pt_new)
            id_old.new_pos(pt_new)
            rep[id_old] = id_old.id

    for id in to_assin:
        if id not in new_pos_traite:
            new_id = Identity(id, conf)
            rep[new_id] = new_id.id
            # print ("\n")
            # print("New element ! ! ! " + str(rep))
            # print("oldpos : " + str(oldPos))
            # print(to_assin)
            # print("Frame : " + str(frame))
            new_pos_traite.append(id)

    for id in oldPos.keys():
        if id not in old_pos_traite:
            rep[id] = oldPos[id]
            old_pos_traite.append(id)
    
    return rep


def dist((A, B)):
    return A.dist(B)


res = make_pred(row_data, make_alea(0.001, dist))
print(res.shape)
print(compute_clear(data_ref, res, 1))


(7663, 39)


(0.9915748541801686, 0.0, 0.5316538310925467, 0.5265068049254699)


In [4]:
import inspect

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold



class InertieClasificator(BaseEstimator, ClassifierMixin):
    def __init__(self, coef_ine_hd = 1, coef_ine_bd =1, coef_dist_hd = 1, coef_dist_bd = 1, 
                 coef_size_hd = 1, coef_size_bd = 1, coef_pt_cache = 10, mem_length= 20):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        for arg, val in values.items():
            setattr(self, arg, val)
        self.conf = values
        
    
    def fit(self, X, y=None):
        # on n'apprend rien pour l'instant
        pass
    
    def predict(self, X, y=None):
        # prediction
        return [make_pred(vid, make_alea(0.01, dist, self.conf)) for vid in X]
    
    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        res = 0
        for vid, raw in zip(pred, y):
            c = compute_clear(raw, vid, 1)
            res += c[0]
        # print(res/len(y))
        return res/len(y)
    
    def get_params(self, deep=True):
        return self.conf
    
    def set_params(self, **params):
        self.conf = params
        return self
        
        

#### Test de la class

In [5]:

est = InertieClasificator(coef_pt_cache= 10, coef_dist_hd= 2.1, coef_dist_bd= 3, coef_size_bd=5, coef_size_hd=6, coef_ine_bd=4, coef_ine_hd= 8, mem_length=50)
print(est.score([data], [data_ref]))


NameError: name 'data' is not defined

In [7]:
data_ref, data = mult_load(["data/Scenario03-02/", "data/Scenario01-01/", "data/Scenario02-03/", "data/Scenario02-01/",
                            "data/Scenario02-02/", "data/Scenario03-03/", "data/Scenario03-02/", "data/Scenario03-01/",
                            "data/Scenario03-05/"])
print(data_ref.shape)

(9,)


In [9]:
est = InertieClasificator(coef_pt_cache= 20, coef_dist_bd=5, coef_dist_hd=5, coef_ine_hd= 5,
                          coef_ine_bd=20, coef_size_bd=1, coef_size_hd=0, mem_length=30)
est2 = InertieClasificator()
print(est2.score(data, data_ref))

0.9800697262180686


In [12]:
dict_test = {"coef_pt_cache" : [10, 20, 30, 50], "coef_size_hd":[0,1], "coef_size_bd":[0,1],
             "coef_ine_hd":[20,25,30,40,50, 60, 70], "coef_ine_bd":[10, 15, 20,25,30, 40, 50],"coef_dist_bd":[0.5, 1,3,5,10], "coef_dist_hd":[0, 0.5, 1,5,10], 
             "mem_length" :[20, 30,40, 50, 70,90], "exposant": [1,0.7,1.3,0.5,0.3]}
test_exp = {"exposant": [1, 0.7,1.3,2,0.5]}
est = InertieClasificator()
gs = RandomizedSearchCV(est, dict_test, n_jobs=4, n_iter=50, verbose = 3, return_train_score=True, cv=KFold(n_splits=2, random_state=1))
gs.fit(data, data_ref)
print("best params : " + str(gs.best_params_))
print("best score : " + str(gs.best_score_))

Fitting 2 folds for each of 50 candidates, totalling 100 fits


[CV] mem_length=90, coef_dist_bd=10, coef_pt_cache=20, coef_ine_bd=25, coef_dist_hd=5, coef_size_bd=1, coef_size_hd=0, coef_ine_hd=30, exposant=0.7 


[CV] mem_length=90, coef_dist_bd=10, coef_pt_cache=20, coef_ine_bd=25, coef_dist_hd=5, coef_size_bd=1, coef_size_hd=0, coef_ine_hd=30, exposant=0.7 


[CV] mem_length=70, coef_dist_bd=0.5, coef_pt_cache=20, coef_ine_bd=40, coef_dist_hd=1, coef_size_bd=0, coef_size_hd=1, coef_ine_hd=20, exposant=1.3 


[CV] mem_length=70, coef_dist_bd=0.5, coef_pt_cache=20, coef_ine_bd=40, coef_dist_hd=1, coef_size_bd=0, coef_size_hd=1, coef_ine_hd=20, exposant=1.3 


...

[CV] mem_length=40, coef_dist_bd=0.5, coef_pt_cache=10, coef_ine_bd=40, coef_dist_hd=0.5, coef_size_bd=1, coef_size_hd=0, coef_ine_hd=30, exposant=0.3 


[CV]  mem_length=30, coef_dist_bd=1, coef_pt_cache=30, coef_ine_bd=15, coef_dist_hd=10, coef_size_bd=1, coef_size_hd=0, coef_ine_hd=30, exposant=0.3, score=0.989937033813, total=  15.3s


[CV] mem_length=40, coef_dist_bd=0.5, coef_pt_cache=10, coef_ine_bd=40, coef_dist_hd=0.5, coef_size_bd=1, coef_size_hd=0, coef_ine_hd=30, exposant=0.3 


[CV] mem_length=40, coef_dist_bd=0.5, coef_pt_cache=50, coef_ine_bd=10, coef_dist_hd=5, coef_size_bd=1, coef_size_hd=1, coef_ine_hd=40, exposant=0.5 


KeyboardInterrupt: 

In [12]:
print("best params : " + str(gs.best_params_))
print("best score : " + str(gs.best_score_))
res = gs.cv_results_["mean_test_score"]
res.sort()
print(res)

best params : {'mem_length': 90, 'coef_dist_bd': 1, 'coef_dist_hd': 0, 'coef_ine_bd': 25, 'coef_ine_hd': 60, 'coef_size_bd': 0, 'coef_size_hd': 0, 'coef_pt_cache': 50, 'exposant': 0.3}
best score : 0.9812315271398112
[0.98064799 0.9806516  0.98066239 0.98066239 0.98066239 0.98066239
 0.98066239 0.98066239 0.98066239 0.98068534 0.98068923 0.98068923
 0.98068923 0.98068923 0.98069363 0.98069363 0.98069363 0.98069725
 0.98069725 0.98069725 0.98069725 0.98070363 0.98070363 0.98070363
 0.98070804 0.98070804 0.98070804 0.98070804 0.98070804 0.98070804
 0.98070804 0.98070804 0.98070804 0.98070804 0.98070804 0.98070804
 0.98070804 0.98070804 0.98070804 0.98071244 0.98072244 0.98072244
 0.98072244 0.98072244 0.98072244 0.98072244 0.98072244 0.98073098
 0.98073098 0.98073098 0.98073098 0.98073098 0.98073098 0.98073098
 0.98073098 0.98073098 0.98073098 0.98073098 0.98073098 0.98073098
 0.98073098 0.98073098 0.98073098 0.98073487 0.98074125 0.98074538
 0.98074538 0.98074538 0.98074538 0.98074538 0

2)best params : {'mem_length': 10, 'coef_dist_bd': 10, 'coef_dist_hd': 10, 'coef_ine_bd': 5, 'coef_ine_hd': 20, 'coef_size_bd': 0, 'coef_size_hd': 0, 'coef_pt_cache': 20}
best score : 0.9784799777087765
3)best params : {'mem_length': 50, 'coef_dist_bd': 1, 'coef_dist_hd': 15, 'coef_ine_bd': 20, 'coef_ine_hd': 20, 'coef_size_bd': 0, 'coef_size_hd': 0, 'coef_pt_cache': 30}
best score : 0.9784799777087765

4)best params : {'mem_length': 50, 'coef_dist_bd': 3, 'coef_dist_hd': 15, 'coef_ine_bd': 20, 'coef_ine_hd': 20, 'coef_size_bd': 0, 'coef_size_hd': 0, 'coef_pt_cache': 30}
best score : 0.9811402467034908

best params : {'mem_length': 70, 'coef_dist_bd': 3, 'coef_dist_hd': 1, 'coef_ine_bd': 25, 'coef_ine_hd': 30, 'coef_size_bd': 0, 'coef_size_hd': 0, 'coef_pt_cache': 20}
best score : 0.981162940034187

best params : {'mem_length': 70, 'coef_dist_bd': 3, 'coef_dist_hd': 1, 'coef_ine_bd': 20, 'coef_ine_hd': 50, 'coef_size_bd': 0, 'coef_size_hd': 0, 'coef_pt_cache': 20, 'exposant': 0.7}
best score : 0.9812315271398112

best params : {'mem_length': 90, 'coef_dist_bd': 1, 'coef_dist_hd': 0, 'coef_ine_bd': 25, 'coef_ine_hd': 60, 'coef_size_bd': 0, 'coef_size_hd': 0, 'coef_pt_cache': 50, 'exposant': 0.3}
best score : 0.9812315271398112


In [7]:
# sauvegarde les donnée
save_data(res, PATH+"res_tmp.txt")

Test avec couleur

In [6]:
from random import random, randint
from munkres import Munkres, print_matrix

def make_alea(proba_trans, conf=None):
    def alea_par_func(data):
        save_pos = dict()
        parle = 0
        for frame in iter_frame(data):
            nb_personne = frame.shape[0]

            if random() < proba_trans:
                parle = randint(0, nb_personne)

            save_pos = head_assignation(frame, save_pos, conf)

            for i, _ in enumerate(frame):
                cp = CorpCol(_)
                # if save_pos[cp] == 5:
                    # print(save_pos)
                yield (save_pos[cp], save_pos[cp] == parle)
    return alea_par_func


def head_assignation(frame, oldPos, conf):
    to_assin = []
    new_pos = []
    rep = dict()
    for line in frame:
        to_assin.append(CorpCol(line))
        new_pos.append(0)
        
    new_pos_traite = []
    old_pos_traite = []
    
    m = Munkres()
    matrice = []
    old_pos_order = oldPos.keys()
    for old in oldPos:
        l =[]
        for new in to_assin:
            l.append(dist((old, new)))
        matrice.append(l)
    # print("old pos oder" + str(old_pos_order))
    # print("to_assin" + str(to_assin))
    # print(matrice)
    # 
    # 
    # if len(matrice) != 0 and (max([max(l)for l in matrice])-min([min(l)for l in matrice]) < 100):
    #     print (frame)
    #     print("max " + str(max([max(l)for l in matrice])))
    #     print("min " + str(min([min(l)for l in matrice])))

    
    if len(matrice) != 0:
        indice = m.compute(matrice)
        i_min = min((len(old_pos_order), len(to_assin)))
        indice = indice[:i_min] # on suprime le 0 pading
        for (old, new) in indice:
            id_old = old_pos_order[old]
            pt_new = to_assin[new]
            old_pos_traite.append(id_old)
            new_pos_traite.append(pt_new)
            id_old.new_pos(pt_new)
            rep[id_old] = id_old.id

    for id in to_assin:
        if id not in new_pos_traite:
            new_id = Identity(id, conf)
            rep[new_id] = new_id.id
            # print ("\n")
            # print("New element ! ! ! " + str(rep))
            # print("oldpos : " + str(oldPos))
            # print(to_assin)
            # print("Frame : " + str(frame))
            new_pos_traite.append(id)

    for id in oldPos.keys():
        if id not in old_pos_traite:
            rep[id] = oldPos[id]
            old_pos_traite.append(id)
    
    return rep


def dist((A, B)):
    return A.dist_col(B)


res = make_pred(row_data, make_alea(0.001))
print(res.shape)
print(compute_clear(data_ref, res, 1))


(7663, 39)


(0.9915748541801686, 0.0, 0.6312491841796111, 0.6254050550874919)
